## **07-2) 심층 신경망**

인공 신경망에 층을 여러 개 추가하여 패션 MNIST 데이터셋을 분류하면서 케라스로 심층 신경망을 만드는 방법을 자세히 배운다.

#### **용어 정리**
- **심층 신경망**: 2개 이상의 층을 포함한 신경망. 종종 다층 인공 신경망, 심층 신경망, 딥러닝을 같은 의미로 사용한다.

- **렐루 함수**: 이미지 분류 모델의 은닉층에 많이 사용하는 활성화 함수이다. 시그모이드 함수는 층이 많을수록 활성화 함수의 양쪽 끝에서 변화가 작기 때문에 학습이 어려워진다. 렐루 함수는 이런 문제가 없고 계산도 간단하다.

- **옵티마이저**: 신경망의 가중치와 절편을 학습하기 위한 알고리즘 또한 방법을 말한다. 케라스에는 다양한 경사 하강법 알고리즘이 구현되어 있으며, 대표적으로 SGD, 네스테로프 모멘텀, RMSprop, Adam 등이 있다.

<br><hr><br>

### **2개의 층**

다시 케라스 API를 사용해서 패션 MNIST 데이터셋을 불러온다.

In [1]:
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [3]:
from sklearn.model_selection import train_test_split

# 이미지의 픽셀값을 0~255 -> 0~1로 변환
train_scaled = train_input / 255.0
# 28*28 크기의 2차원 배열 ~> 784 크기의 1차원 배열
train_scaled = train_scaled.reshape(-1, 28*28)
# 훈련 세트와 검증 세트 분리
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

이제 인공 신경망 모델이 층을 2개 추가해볼 것이다.<br>
입력층과 출력층 사이에 밀집층을 추가할 건데, 이렇게 입력층과 출력층 사이에 있는 모든 층을 **은닉층**(hidden layer)이라고 한다.

모든 신경망의 은닉층에는 항상 활성화 함수가 있는데, **활성화 함수**는 신경망 층의 선형방정식의 계산 값에 적용하는 함수이다.<br>
출력층에 적용하는 활성화 함수는 종류가 제한되어 있지만(이진 분류일 경우 시그모이드, 다중 분류일 경우 소프트맥스),<br>
은닉층의 활성화 함수는 비교적 자유롭다.


많이 사용하는 활성화 함수 중 하나는 시그모이드 함수이다. 이 함수는 뉴런의 출력값을 0 ~ 1 사이로 압축한다.<br><br>
시그모이드 활성화 함수를 사용한 은닉층과 소프트맥스 함수를 사용한 출력층을 케라스의 Dense 클래스로 만들어보자.<br>
케라스에서 신경망의 첫 번째 층은 input_shape 매개변수로 입력의 크기를 꼭 지정해줘야 한다.

In [4]:
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)) # 은닉층, 100개의 뉴런을 가진 밀집층
dense2 = keras.layers.Dense(10, activation='softmax') # 출력층, 10개의 뉴런(10개의 클래스를 분류하므로)

은닉층의 뉴런 개수를 정하는 데는 특별한 기준이 없다. 몇 개의 뉴런을 둬야 할지 판단하기 위해서는 상당한 경험이 필요하다.<br>
한 가지 제약사항은 은닉층의 뉴런 개수는 출력층보다 많아야 한다는 것이다. 클래스 10개에 대한 확률을 예측해야 하는데 이전 은닉층의 뉴런이 10개보다 적으면 부족한 정보가 전달될 것이다.

<br><hr><br>

### **심층 신경망 만들기**

dense1과 dense2 객체를 Sequential 클래스에 추가하여 **심층신경망**(DNN; Deep Neural Network)을 만들어 볼 것이다.

In [5]:
model = keras.Sequential([dense1, dense2]) 

Sequential 클래스의 객체를 만들 때 여러 층을 추가하려면 리스트로 만들어 전달해야 한다.<br>
주의할 점은 출력층을 가장 마지막에 두어야 한다는 것이다.

케라스는 모델의 summary() 메서드를 호출하여 층에 대한 유용한 정보를 얻을 수 있다.

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


- 첫 번째 줄에 모델의 이름이 출력되고, 그다음 이 모델에 들어있는 층이 순서대로 나열된다.
- 층마다 층 이름, 클래스, 출력 크기, 모델 파라미터 개수가 출력된다.
- 층을 만들 때 name 매개변수로 이름을 지정할 수 있는데, 지정하지 않으면 자동으로 'dense'라고 이름을 붙인다.

**Output Shape**
- 출력 크기를 보면 (None, 100)인데, 첫 번째 차원은 **샘플의 개수**를 나타낸다. 샘플 개수가 아직 정의되어 있지 않아 None인데, 케라스 모델의 fit() 메서드에 훈련 데이터를 주입하면 미니배치 경사 하강법을 수행하기 때문이다.
- 케라스의 기본 미니배치 크기는 32개이며, fit() 메서드에서 batch_size 매개변수로 지정할 수 있다.
- 따라서 샘플 개수를 고정하지 않고 어떤 배치 크기에도 유연하게 대응할 수 있도록 None으로 설정한다.
- 이렇게 신경망 층에 입력되거나 출력되는 배열의 첫 번째 차원을 **배치 차원**이라고 한다.

- 두 번째 차원인 100은 은닉층의 뉴런 개수를 100개로 두었으니 100개의 출력이 나온다는 의미이다. 즉 샘플마다 784개의 픽셀값이 은닉층을 통과하며서 100개의 특성으로 압축되었다.

**모델 파라미터 개수**
- 이 층은 Dense 층이므로 입력 픽셀 784개와 100개의 모든 조합에 대한 가중치가 있다. 또 뉴런마다 1개의 절편이 있으므로 모델 파라미터는 `784 * 100 + 100 = 78500`개이다.

summary() 메서드 마지막에는 총 모델 파라미터 개수와 훈련되는 파라미터 개수가 동일하게 79,510개로 나온다. <br>
은닉층과 출력층의 파라미터 개수를 합친 값이다.

그 아래 훈련되지 않는 파라미터는 0으로 나온다. 간혹 경사하강법으로 훈련되지 않는 파라미터를 가진 층이 여기에 나타난다.

<br><hr><br>

### **층을 추가하는 다른 방법**

#### **1. Sequential 클래스 생성자 안에서 바로 Dense 클래스의 객체 생성**

In [7]:
model = keras.Sequential([keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
                          keras.layers.Dense(10, activation='softmax', name='output')], name='패션 MNIST 모델')

- 이렇게 작업하면 추가되는 층을 한눈에 쉽게 알아볼 수 있다.
- name 매개변수로 모델과 각 층의 이름을 지정할 수 있는데, 모델과 달리 층의 이름은 반드시 영문이어야 한다.

In [8]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


이 방법이 편리하지만 아주 많은 층을 추가하려면 Sequential 클래스 생성자가 매우 길어지며, 조건에 따라 층을 추가할 수도 없다.

#### **2. 모델의 add() 메서드 사용**

Sequential 클래스에서 층을 추가할 때 가장 널리 사용하는 방법이다.

In [9]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

한눈에 추가되는 층을 볼 수 있고, 프로그래 실행 시 동적으로 층을 선택하여 추가할 수 있다.

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


이제 모델을 훈련해보자.

In [12]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5633 - accuracy: 0.8076
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4063 - accuracy: 0.8540
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3718 - accuracy: 0.8675
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3492 - accuracy: 0.8740
Epoch 5/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3329 - accuracy: 0.8810


추가된 층이 성능을 향상시킨 것을 알 수 있다.

<br><hr><br>

### **렐루(ReLU) 함수**

시그모이드 함수는 오른쪽과 왼쪽 끝으로 갈수록 그래프가 누워있기 때문에 올바른 출력을 만드는데 신속하게 대응하지 못한다.<br>
특히 층이 많은 심층 신경망일수록 그 효과가 누적되어 학습을 더 어렵게 만든다.<br>
이를 개선하기 위한 다른 종류의 활성화 함수가 **렐루 함수**이다.

렐루 함수는 입력이 양수일 경우 마치 활성화 함수가 없는 것처럼 그냥 입력을 통과시키고, 음수일 경우에는 0으로 만든다.<br><br>

max(0,z)와 같이 작성할 수 있다.<br>
z가 0보다 크면 z를, 0보다 작으면 0을 출력한다는 의미이다.

렐루 함수는 이미지 처리에서 특히 좋은 성능을 낸다.

은닉층의 활성화 함수에 렐루 함수를 적용해보자.<br>
여기서는 reshape() 메서드로 데이터셋을 직접 1차원으로 펼치는 대신 케라스의 **Flatten 층**을 이용할 것이다.

In [13]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))
# +) 이 신경망의 깊이는 여전히 2이다. Flatten 클래스는 학습하는 층이 아니므로 제외한다.

In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 100)               78500     
                                                                 
 dense_5 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


케라스의 Flatten 층을 신경망 모델에 추가하면 입력값의 차원을 짐작할 수 있다는 장점이 있다.<br>
앞의 출력에서 784개의 입력이 첫 번째 은닉층에 전달된다는 것을 알 수 있다.

그럼 훈련 데이터를 다시 준비해서 모델을 훈련해보자.

In [17]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [18]:
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5312 - accuracy: 0.8109
Epoch 2/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3932 - accuracy: 0.8579
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3537 - accuracy: 0.8719
Epoch 4/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3349 - accuracy: 0.8795
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3192 - accuracy: 0.8844


시그모이드 함수를 사용했을 때보다 성능이 조금 더 향상되었다.

그럼 검증 세트에서의 성능도 확인해보자.

In [19]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 2s 3ms/step - loss: 0.3799 - accuracy: 0.8780


[0.3798533082008362, 0.878000020980835]

은닉층을 추가하지 않은 경우보다 몇 퍼센트 성능이 향상되었다.

<br><hr><br>

### **옵티마이저**

**옵티마이저**는 신경망의 가중치와 절편을 학습하기 위한 알고리즘 또한 방법이다.

가장 기본적인 옵티마이저는 확률적 경사 하강법인 SGD이다.<br>
이름이 SGD이지만 1개의 샘플을 뽑아서 훈련하지 않고서 앞서 언급한 것처럼 기본적으로 미니배치를 사용한다.

SGD 옵티마이저를 사용하려면 compile() 메서드의 optimizer 매개변수를 'sgd'로 지정하면 된다.

- 방법 1
```
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')
```
- 방법 2
```
# 이 옵티마이저는 tensorflow.keras.optimizers 패키지 아래
# SGD 클래스로 구현되어 있다.
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics='accuracy')
```


만약 SGD 클래스의 학습률 기본값이 0.01일 때 이를 바꾸고 싶으면 다음과 같이 learning_rate 매개변수에 원하는 학습률을 지정하면 된다.
```
sgd = keras.optimizers.SGD(learning_rate=0.1)
```

SGD 외에도 다양한 옵티마이저들이 있다.

**기본 경사 하강법 옵티마이저**

- 기본 경사 하강법 옵티마이저는 모두 SGD 클래스에서 제공한다.

- SGD 클래스의 **momentum 매개변수**의 기본값은 0이다. 0보다 크게 지정하면 마치 이전의 그레이디언트를 가속도처럼 사용하는 **모멘텀 최적화**를 사용한다.
보통 momentum은 0.9 이상으로 지정한다.

- SGD 클래스의 **nesterov 매개변수**를 기본값 False에서 True로 바꾸면 **네스테로프 모멘텀 최적화(=네스테로프 가속 경사)**를 사용한다. 네스테로프 모멘텀은 모멘텀 최적화를 2번 반복하여 구현한다. 대부분의 경우 네스테로프 모멘텀 최적화가 기본 확률적 경사 하강법 보다 나은 성능을 제공한다.

**적응적 학습률 옵티마이저**

- 모델이 최적점에 가까이 갈수록 학습률을 낮출 수 있다. 이렇게 하면 안정적으로 최적점에 수렴할 가능성이 높은데, 이러한 학습률을 **적응적 학습률**(adaptive learning rate)이라고 한다. 학습률 매개변수를 튜닝하는 수고를 덜 수 있다는 장점이 있다.

- 대표적인 옵티마이저는 **Adagrad**와 **RMSprop**이다. 각각 optimizer 매개변수에 'adagrad', 'rmsprop'으로 지정하면 된다. optimizer의 기본값은 'rmsprop'이다. 옵티마이저 사용법은 sgd를 사용할 때와 같다.

- **Adam**은 모멘텀 최적화와 RMSprop의 장점을 접목한 것이다.

- 적응적 학습률을 사용하는 이 3개의 클래스는 learning_rate 매개변수의 기본값으로 모두 0.001을 사용한다.

<br><br>

여기서는 Adam 클래스의 매개변수 기본값을 사용해 패션 MNIST 모델을 훈련해볼 것이다.

In [21]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28,28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [22]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5268 - accuracy: 0.8155
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3912 - accuracy: 0.8605
Epoch 3/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3514 - accuracy: 0.8726
Epoch 4/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3257 - accuracy: 0.8812
Epoch 5/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3046 - accuracy: 0.8880


In [23]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3377 - accuracy: 0.8821


[0.3377113342285156, 0.8820833563804626]